In [1]:
%reset -f

In [1]:
#Preprocessing
import sys
sys.path = ['/nfs/gns/homes/willj/anaconda3/envs/GTEx/lib/python3.5/site-packages'] + sys.path
import h5py
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib_venn import venn3, venn3_circles
import pickle
import gzip
import pandas as pd
import requests
import openslide
from openslide.deepzoom import DeepZoomGenerator
from openslide import open_slide
import math
import pdb
import time
import os
import matplotlib.patches as mpatches

# Functions

In [4]:
tissue_types = ['Lung', 'Artery - Tibial', 'Heart - Left Ventricle', 'Breast - Mammary Tissue', 'Brain - Cerebellum', 'Pancreas', 'Testis', 'Liver', 'Ovary', 'Stomach']

genotypes_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/GenotypeFiles/phg000520.v2.GTEx_MidPoint_Imputation.genotype-calls-vcf.c1/parse_data/GTEx_Analysis_20150112_OMNI_2.5M_5M_450Indiv_chr1to22_genot_imput_info04_maf01_HWEp1E6_ConstrVarIDs_all_chrom_filered_maf_subset_individuals_44_tissues.hdf5'
expression_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/ExpressionFiles/phe000006.v2.GTEx_RNAseq.expression-data-matrixfmt.c1/parse_data/44_tissues/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm_*_normalised_without_inverse_gene_expression.txt'
phenotype_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/PhenotypeFiles/phs000424.v6.pht002743.v6.p1.c1.GTEx_Sample_Attributes.GRU.txt.gz'

def build_empty_model():
    inception_model = InceptionV3(weights='imagenet', include_top=False)

    x = inception_model.output

    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    model = Model(input=inception_model.input, output=predictions)
    return model

# Load data

In [3]:
data_tile_number = 10
data_tile_level_index = -1
raw_X = []
tissue_labels = []
IDs = []
for t in tissue_types:
    print (t)
    [rX, tl, ID] = pickle.load(open('../data/processed/patches/data_{}_{}_{}.py'.format(t,data_tile_number,data_tile_level_index), 'rb'))
    raw_X.extend(rX)
    tissue_labels.extend(tl)
    IDs.extend(ID)
X = np.array(raw_X)

NameError: name 'tissue_types' is not defined

# Extract genotypes

In [4]:
def get_donor_IDs(IDlist):
    return [str(x).split('-')[1] for x in IDlist]

with h5py.File(genotypes_filepath,'r') as f:
    sampleIDs = list(f['genotype']['row_header']['sample_ID'])
    genotype_matrix = np.array(f['genotype']['matrix'])
    genotype_matrix[genotype_matrix == 255] = 0

# Get donor IDs of the images

In [5]:
donor_IDs = get_donor_IDs(sampleIDs)
image_donor_IDs = get_donor_IDs(IDs)

# Filter to genotypes we actually have

In [6]:
genotype_exists = np.array([x in donor_IDs for x in image_donor_IDs])
print (sum(genotype_exists))

8430


In [7]:
f_X = X[genotype_exists]
f_tissue_labels = np.array(tissue_labels)[genotype_exists]
f_IDs = np.array(IDs)[genotype_exists]
print (len(f_X), len(f_tissue_labels), len(f_IDs))

8430 8430 8430


In [8]:
f_image_donor_IDs = get_donor_IDs(f_IDs)
#Now all image donors have genotypes
assert all([x in donor_IDs for x in f_image_donor_IDs]) == True

# Load representations

In [9]:
model_tile_number = 50
model_tile_level_index = -1

full_representations = pickle.load(open('../data/processed/representations/representations_model_{}_{}_gs{}_data_{}_{}'.format(model_tile_number,model_tile_level_index,1,data_tile_number,data_tile_level_index), 'rb'))
f_full_representations = full_representations[genotype_exists]
f_donor_IDs = get_donor_IDs(f_IDs)

# Extract necessary matrix rows in correct order from genotype matrix

In [11]:
genotype_matrix.shape

(449, 10727091)

In [14]:
sys.getsizeof(X)

5299691424

In [13]:
sys.getsizeof(genotype_matrix)

4816463971

In [ ]:
sample_idx = np.array([donor_IDs.index(x) for x in f_donor_IDs])

In [ ]:
f_genotype_matrix = genotype_matrix[sample_idx, :]
print (f_genotype_matrix.shape)

# Example linear associations with lim

In [20]:
from lim.tool import normalize
from lim.genetics import qtl
from lim.genetics.phenotype import NormalPhenotype

from numpy import random
from numpy import ones
from numpy import zeros
from numpy import sqrt
from numpy import ones

random = random.RandomState(0)
N = 50
P = 100

# genetic markers
X = random.randn(N, P)
X = normalize.stdnorm(X, axis=0)
X /= sqrt(X.shape[1])

# effect sizes
u = zeros(P)
u[0] = +1
u[1] = +1
u[2] = -1
u[3] = -1
u[4] = -1
u[5] = +1

offset = 0.4

# phenotype definition
y = offset + X.dot(u) + 0.2 * random.randn(N)

G = X[:, 2:].copy()

lrt = qtl.scan(NormalPhenotype(y), X, G, progress=False)
print(lrt.pvalues())

INFO:lim.genetics.qtl._scan:Normal association scan has started.
INFO:lim.genetics.qtl._scan:Number of candidate markers to scan: 100
INFO:lim.genetics.qtl._scan:Genetic markers normalization.
INFO:lim.genetics.qtl._scan:Computing the economic eigen decomposition.
INFO:lim.genetics.qtl._scan:Genetic marker candidates normalization.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.


[ 0.0059433   0.02481828  0.01364253  0.00796372  0.01665129  0.0139813
  0.99797416  0.82462016  0.10035177  0.94504485  0.54028934  0.60235573
  0.58967876  0.80357017  0.7430283   0.88782955  0.45937029  0.65233407
  0.4631676   0.35344608  0.39094646  0.66382861  0.70909249  0.93160763
  0.59810102  0.14507473  0.10088497  0.00967413  0.24217988  0.6726014
  0.35020917  0.75734146  0.4376674   0.94870166  0.88362786  0.92749008
  0.30058976  0.34372389  0.15346698  0.10287649  0.02084949  0.17561226
  0.74867121  0.89208037  0.58584088  0.17302564  0.8044232   0.47024474
  0.34775072  0.25478353  0.6515139   0.18032672  0.35264529  0.81173092
  0.11153409  0.81657815  0.4729755   0.76140385  0.12763198  0.25481329
  0.15908389  0.40858544  0.1714145   0.38068986  0.77456634  0.97351764
  0.15354026  0.12652926  0.2886353   0.2691533   0.63232665  0.88815939
  0.33865953  0.53291733  0.38837006  0.98472595  0.30993077  0.92855554
  0.3519052   0.53092571  0.61686339  0.93070914  0.5

# Finding Linear associations

In [13]:
print (f_full_representations.shape)
print (f_genotype_matrix.shape)

(861, 1024)
(861, 10727091)


In [14]:
from lim.genetics import qtl
from lim.genetics.phenotype import NormalPhenotype

In [62]:
n_samples = 600
X_matrix = f_genotype_matrix[:,0:1000]
y_pheno = f_full_representations[:,0]

N = f_genotype_matrix.shape[0]
P = f_genotype_matrix.shape[1]
y_pheno_obj = NormalPhenotype(y_pheno)

G = X_matrix[:, 1:].copy()

lrt = qtl.scan(y_pheno_obj, X_matrix, G, progress=False)

print(min(lrt.pvalues()))

INFO:lim.genetics.qtl._scan:Normal association scan has started.
INFO:lim.genetics.qtl._scan:Number of candidate markers to scan: 1000
INFO:lim.genetics.qtl._scan:Genetic markers normalization.
INFO:lim.genetics.qtl._scan:Computing the economic eigen decomposition.
INFO:lim.genetics.qtl._scan:Genetic marker candidates normalization.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.
INFO:lim.genetics.qtl._qtl:Computing likelihood-ratio test statistics.


0.000105358920954


In [ ]:
lr